In [9]:
import numpy as np
import pandas as pd

from tqdm import tqdm, tqdm_notebook

from scipy import stats
from sklearn.model_selection import GroupKFold

import torch
import torch.nn as nn
from torch.nn import functional as F
import torch.utils.data
from transformers import *

import os
import re
import math
import random
from matplotlib import pyplot as plt
import warnings
from math import floor, ceil

warnings.filterwarnings('ignore')
device = torch.device('cuda')
#device = torch.device('cpu')
torch.backends.cudnn.benchmark=True

%matplotlib inline

# Read Data  & Set Path

In [10]:
train = pd.read_csv('C:/Users/Lab000/Desktop/kaggle/kaggle_competetion/Google_Quest_LABEL/my-solution/input/google-quest-challenge/train.csv').fillna(' ')
sub = pd.read_csv('C:/Users/Lab000/Desktop/kaggle/kaggle_competetion/Google_Quest_LABEL/my-solution/input/google-quest-challenge/sample_submission.csv').fillna(' ')
output_dir="./double/token_model_config/"

# Function Definition

In [11]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

def spearman_corr(y_true, y_pred):
    if np.ndim(y_pred) == 2:
        corr = np.nan_to_num([stats.spearmanr(y_true[:, i], y_pred[:, i])[0] for i in range(y_true.shape[1])]).mean()
    else:
        corr = stats.spearmanr(y_true, y_pred)[0]
    return corr
  
def calc_each_spearman(valid_y, valid_pred):
    lst = []
    for idx in range(30):
        spearman = spearman_corr(valid_y[:,idx], valid_pred[:,idx])
        lst.append(spearman)
    df = pd.DataFrame(lst).T
    df.columns = class_names3
    return df

In [12]:
def _get_segments(tokens, max_seq_length):
    """Segments: 0 for the first sequence, 1 for the second"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    segments = []
    first_sep = True
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            if first_sep:
                first_sep = False
                current_segment_id = 1#新增 
            else:
                current_segment_id = 1
    return segments + [0] * (max_seq_length - len(tokens))

def _get_ids(tokens, tokenizer, max_seq_length):
    """Token ids from Tokenizer vocab"""
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_ids = token_ids + [0] * (max_seq_length-len(token_ids))
    return input_ids

def _trim_input(title, question, answer, max_sequence_length=512-1, 
                t_max_len=70-1, q_max_len=219, a_max_len=219):#???

    t = tokenizer.tokenize(title)
    q = tokenizer.tokenize(question)
    a = tokenizer.tokenize(answer)
    
    t_len = len(t)
    q_len = len(q)
    a_len = len(a)

    if (t_len+q_len+a_len+4) > max_sequence_length:
        
        if t_max_len > t_len:
            t_new_len = t_len
            a_max_len = a_max_len + floor((t_max_len - t_len)/2)
            q_max_len = q_max_len + ceil((t_max_len - t_len)/2)
        else:
            t_new_len = t_max_len
      
        if a_max_len > a_len:
            a_new_len = a_len 
            q_new_len = q_max_len + (a_max_len - a_len)
        elif q_max_len > q_len:
            a_new_len = a_max_len + (q_max_len - q_len)
            q_new_len = q_len
        else:
            a_new_len = a_max_len
            q_new_len = q_max_len
            
            
        if t_new_len+a_new_len+q_new_len+4 != max_sequence_length:
            raise ValueError("New sequence length should be %d, but is %d" 
                             % (max_sequence_length, (t_new_len+a_new_len+q_new_len+4)))

        t = t[:t_new_len]
        q = norm_token_length(q, q_new_len)
        a = norm_token_length(a, a_new_len)
    
    return t, q, a

def norm_token_length(tokens, l):
    if len(tokens) > l:
        head = l//2
        tail = l - head
        return tokens[:head] + tokens[-tail:]
    else:
        return tokens[:l]

def _convert_to_bert_inputs(title, question, answer, cate, max_sequence_length=512):
    """Converts tokenized input to ids, masks and segments for BERT"""
    #stoken = ["[CLS]"] + [cate] + title + ["[SEP]"] + question + ["[SEP]"] + answer + ["[SEP]"]
    stoken_1 = ["[CLS]"] + [cate] + title + ["[SEP]"] + question + ["[SEP]"]
    #stoken_2 = ["[CLS]"] + title + ["[SEP]"] + answer + ["[SEP]"]#version1
    stoken_2 = ["[CLS]"] + [cate]+ title + ["[SEP]"] + answer + ["[SEP]"]
    input_ids = _get_ids(stoken_1, tokenizer, max_sequence_length)
    input_ids_2 = _get_ids(stoken_2, tokenizer, max_sequence_length)
    input_segments = _get_segments(stoken_1, max_sequence_length)
    input_segments_2 = _get_segments(stoken_2, max_sequence_length)
    
    
    return input_ids, input_segments,input_ids_2,input_segments_2

def convert_row(row):
    
    c = f"[{row['category']}]"
    t, q, a = row["question_title"], row["question_body"], row["answer"]
    t, q, a = _trim_input(t, q, a)
    
    ids, segments, ids2, segments2 = _convert_to_bert_inputs(t, q, a, c)
   
    return np.array([[ids, segments, ids2, segments2]])

# Add Category Token

In [13]:
categories = train["category"].unique().tolist()
categories = [f"[{c}]" for c in categories]

In [14]:
model="bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model)
tokenizer.add_tokens(categories)


loading configuration file config.json from cache at C:\Users\Lab000/.cache\huggingface\hub\models--bert-base-cased\snapshots\5532cc56f74641d4bb33641f5c76a55d11f846e0\config.json
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading file vocab.txt from cache at C:\Users\Lab000/.cache\huggingface\hub\models--bert-base-cased\snapshots\5532cc56f74641d4bb33641f5c76a55d11f846e0\vocab.txt
l

5

In [15]:
out_path="./bert_based_tokenizer"
tokenizer.save_pretrained(output_dir+"tokenizer")

tokenizer config file saved in ./double/token_model_config/tokenizer\tokenizer_config.json
Special tokens file saved in ./double/token_model_config/tokenizer\special_tokens_map.json


('./double/token_model_config/tokenizer\\tokenizer_config.json',
 './double/token_model_config/tokenizer\\special_tokens_map.json',
 './double/token_model_config/tokenizer\\vocab.txt',
 './double/token_model_config/tokenizer\\added_tokens.json',
 './double/token_model_config/tokenizer\\tokenizer.json')

# Training Data

In [16]:
%%time
X = train.apply(convert_row, axis=1).values
X = np.vstack(X).reshape((len(X), 2048))
assert X.shape == (6079, 2048)

Token indices sequence length is longer than the specified maximum sequence length for this model (649 > 512). Running this sequence through the model will result in indexing errors


Wall time: 6.99 s


# Label

In [17]:
class_names = list(sub.columns[1:22])
class_names2 = list(sub.columns[22:])
class_names3= list(sub.columns[1:])
y = train[class_names].values#(6079,21)
y2 = train[class_names2].values
y3 = train[class_names3].values
lst = []
#### Opt Binning ，可以參考 C:\Users\Lab000\Desktop\kaggle\kaggle_competetion\Google_Quest_LABEL\26th-solution\KaggleQuest-master\work\Quest_BERT-train.ipynb

# for idx in range(30):
#     t = pd.DataFrame(y[:,idx])[0]
#     # print(len(t))
#     #print(1-t.value_counts())
#     #print(1-t.value_counts()/len(t))
#     w_df = (1-t.value_counts()/len(t)).reset_index()
#     #print(w_df)
#     w_dic = {row["index"]: row[0] for _, row in w_df.iterrows()}
#     # print("=======================")
#     # print(w_dic)
#     w = t.map(w_dic).values#(6079,)
#     #print(w.shape)
#     lst.append(w)
# print(lst)
# print("==================================")
# weights = np.vstack(lst).T#轉置, shape:(6079,30)

#y = np.hstack([y, weights])#(6079,60)

#############################################################################
import copy
y_true = copy.deepcopy(y)
y_true2 = copy.deepcopy(y2)

# Loss Function & Bert Architecture

In [24]:
def custom_loss(data,data2, targets,targets2):
    ############## Opt Binning Loss Calculation####### 
    # mse = nn.MSELoss(reduction="none")(data[:,:30].sigmoid(), targets[:,:30])
    # bce = nn.BCEWithLogitsLoss(reduction='none')(data[:,:30], targets[:,:30])
    # w =  targets[:,30:]
    # loss = (mse*w).sum() + bce.sum()
    #######################################
    
    mse = nn.MSELoss(reduction="none")(data[:,:].sigmoid(), targets[:,:])
    bce = nn.BCEWithLogitsLoss(reduction='none')(data[:,:], targets[:,:])#??
    
    mse2 = nn.MSELoss(reduction="none")(data2[:,:].sigmoid(), targets2[:,:])
    bce2 = nn.BCEWithLogitsLoss(reduction='none')(data2[:,:], targets2[:,:])#??

    loss = (mse).sum()+ bce.sum()+mse2.sum()+bce2.sum()
    return loss

class CustomBert(nn.Module):
    def __init__(self,model,config_path=None):
        super(CustomBert, self).__init__()
        self.config = AutoConfig.from_pretrained(model) 
        
        self.config.Q_labels = 21
        self.config.A_labels = 9
        self.config.output_hidden_states = True
        self.n_use_layer = 4 #原本
        
        self.double_bert= 1
        self.n_labels = self.config.num_labels
        self.config.save_pretrained(output_dir+"config")
        self.bert=AutoModel.from_pretrained(model, config=self.config)
        self.bert2=AutoModel.from_pretrained(model, config=self.config)
        self.bert.save_pretrained(output_dir+"model")
        self.dense1 = nn.Linear(self.double_bert*self.config.hidden_size*self.n_use_layer, self.double_bert*self.config.hidden_size*self.n_use_layer)
        self.dense2 = nn.Linear(self.double_bert*self.config.hidden_size*self.n_use_layer, self.double_bert*self.config.hidden_size*self.n_use_layer)
        self.dropout = nn.Dropout(self.config.hidden_dropout_prob)
        self.classifier = nn.Linear(self.double_bert*self.config.hidden_size*self.n_use_layer, self.config.Q_labels)
        self.classifier2 = nn.Linear(self.double_bert*self.config.hidden_size*self.n_use_layer, self.config.A_labels)
        

    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None
                ,input_ids2=None, attention_mask2=None, token_type_ids2=None,position_ids=None, head_mask=None, inputs_embeds=None, labels=None):

        outputs = self.bert(input_ids,
                            attention_mask=attention_mask,
                            token_type_ids=token_type_ids
                            )
        outputs2 = self.bert2(input_ids2,
                            attention_mask=attention_mask2,
                            token_type_ids=token_type_ids2
                            )
        
        
        pooled_output = torch.cat([outputs[2][-1*(i+1)][:,0] for i in range(self.n_use_layer)], dim=1)#把倒數最後4個layer的cls output concat在一起，把4個(8,768) concat，變成(8,3072) #原本
        pooled_output2 = torch.cat([outputs2[2][-1*(i+1)][:,0] for i in range(self.n_use_layer)], dim=1)
        #pooled_output = torch.cat([outputs[2][-1*(i+1)][:,0] for i in range(self.n_use_layer)], dim=1)#把倒數最後2個layer的cls output concat在一起,把2個(8,768) concat，變成(8,1536)
        #pooled_output2 = torch.cat([outputs2[2][-1*(i+1)][:,0] for i in range(self.n_use_layer)], dim=1)#同上
        
        
        pooled_output = self.dense1(pooled_output)
        pooled_output = self.dense2(pooled_output)
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        pooled_output2 = self.dense1(pooled_output2)
        pooled_output2 = self.dense2(pooled_output2)
        pooled_output2 = self.dropout(pooled_output2)
        logits2 = self.classifier2(pooled_output2)

        outputs = (logits,) + outputs[2:]
        outputs2 = (logits2,) + outputs2[2:]

        return outputs,outputs2

# 檢查參數

In [ ]:
# model=CustomBert(model)
# param_optimizer = list(model.named_parameters())
# #print(param_optimizer[0])
# print("==============================")
# print(param_optimizer[0][0])
# print("==============================")
# print(param_optimizer[0][1])

# 超參數及訓練

In [25]:

N_FOLD=10
N_BERT_LABEL = 30
SEED = 42
#BS = 8
BS = 2
# parameter
n_epoch = 3
learning_rate = 5e-5
max_grad_norm = 1.0

gkf = GroupKFold(n_splits=N_FOLD).split(X=train["question_body"], groups=train["question_body"])#??

spearman_scores = []
best_spearman_lst = []
losses_lst = []
epoch_spearman_lst = []
lr_lst_lst = []
each_speaman_dfs = []

for fold, (train_idx, valid_idx) in enumerate(gkf):#??
  
  if fold in [0,1,2,3]:
    continue

  seed_everything(SEED)

  ### Finetune　###
  # bert_path=f"./DoubleBertBasedCase/bce_no_opt_binning/train/double-bert-based-case_f{fold}_best"
  # model.load_state_dict(torch.load(bert_path),strict=False)
  #################
  model_name="bert-base-cased"
  model=CustomBert(model_name)
  
  model = model.to(device)
  model.bert.resize_token_embeddings(len(tokenizer))
  model.bert2.resize_token_embeddings(len(tokenizer))
  model = model.train()
  
  # optimizer setting
  param_optimizer = list(model.named_parameters())
  no_decay = ['bias', 'LayerNorm.weight', 'LayerNorm.bias']
  optimizer_grouped_parameters = []
  max_lrs = []
  for param in param_optimizer:
    if any(n in param[0] for n in no_decay):#weight_decay
      weight_decay = 0.0
    else:
      weight_decay = 0.1
    if param[0].find("bert.encoder.layer") != -1:
      
      n_diff_last = 11 - int(param[0].split(".")[3])
      lr = learning_rate*0.9**n_diff_last
    elif "embeddings" in param[0]:
      lr = learning_rate*0.9**11
    else:
      lr = learning_rate
    max_lrs.append(lr)
    d = {"params": param[1], "weight_decay": weight_decay}
    optimizer_grouped_parameters.append(d)
  optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate, correct_bias=True)
  
  # train valid split
  train_x = X[train_idx]
  valid_x = X[valid_idx]
  train_y = y[train_idx]
  valid_y = y[valid_idx]

  train_y2 = y2[train_idx]
  valid_y2 = y2[valid_idx]
  valid_y3 = y3[valid_idx]
  # set loader  
  train_dataset = torch.utils.data.TensorDataset(torch.tensor(train_x, dtype=torch.long), torch.tensor(train_y, dtype=torch.float),torch.tensor(train_y2, dtype=torch.float))
  train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BS, shuffle=True)
  valid_dataset = torch.utils.data.TensorDataset(torch.tensor(valid_x, dtype=torch.long), torch.tensor(valid_y, dtype=torch.float),torch.tensor(valid_y2, dtype=torch.float))
  valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=BS, shuffle=False)

  # set schedueler
  num_training_steps = len(train_loader)*n_epoch
  scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=max_lrs, total_steps=num_training_steps)

  model.zero_grad()
  optimizer.zero_grad()
   
  best_spearman = 0
  losses = []
  epoch_spearman = []
  lr_lst = []
  for epoch in range(n_epoch):
    lr = np.array([param_group["lr"] for param_group in optimizer.param_groups]).mean()
    tk0 = tqdm_notebook(enumerate(train_loader), total=len(train_loader), leave=False)
    for i, (x_batch, y_batch,y_batch2) in tk0:
      input_ids = x_batch[:, :512]
      token_ids = x_batch[:, 512:1024]
      input_ids2 = x_batch[:, 1024:1536]
      token_ids2 = x_batch[:, 1536:]
      
      y_pred,y_pred2 = model(input_ids.to(device), attention_mask=(input_ids > 0).to(device), token_type_ids=token_ids.to(device),
                     input_ids2=input_ids2.to(device),attention_mask2=(input_ids2 > 0).to(device),token_type_ids2=token_ids2.to(device))
      #y_pred = model(input_ids.to(device), attention_mask=mask.to(device), token_type_ids=token_ids.to(device))
      loss = custom_loss(y_pred[0], y_pred2[0],y_batch.to(device),y_batch2.to(device))
      loss.backward()
      torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm) 
      optimizer.step()
      optimizer.zero_grad()
      scheduler.step()
      lr_lst.append(np.array([param_group["lr"] for param_group in optimizer.param_groups]).mean())
      losses.append(float(loss))

    # epoch validation
    for param in model.parameters():
      param.requires_grad=False
    model.eval()

    lst = []
    sum_loss = 0
    for i, (x_batch, y_batch,y_batch2)  in enumerate(valid_loader):
      input_ids = x_batch[:, :512]
      token_ids = x_batch[:, 512:1024]
      input_ids2 = x_batch[:, 1024:1536]
      token_ids2 = x_batch[:, 1536:]

      
      with torch.no_grad():
        #y_pred = model(input_ids.to(device), attention_mask=(input_ids > 0).to(device), token_type_ids=token_ids.to(device))
        y_pred,y_pred2 = model(input_ids.to(device), attention_mask=(input_ids > 0).to(device), token_type_ids=token_ids.to(device),
                     input_ids2=input_ids2.to(device),attention_mask2=(input_ids2 > 0).to(device),token_type_ids2=token_ids2.to(device))
        
        loss = custom_loss(y_pred[0], y_pred2[0],y_batch.to(device),y_batch2.to(device))
      #print(y_pred[0].shape)
      
      total_y_pred=torch.cat((y_pred[0],y_pred2[0]),dim=1)
      
      #lst.append(y_pred[0].sigmoid().cpu().squeeze().numpy())
      lst.append(total_y_pred.sigmoid().cpu().squeeze().numpy())
      #lst+(y_pred2[0].sigmoid().cpu().squeeze().numpy())
      sum_loss += loss.cpu().squeeze().numpy()
    valid_pred = np.vstack(lst)#(608,30)
    
    ave_loss = sum_loss/len(valid_loader)

    spearman_score = spearman_corr(valid_y3[:,:N_BERT_LABEL], valid_pred)  
    epoch_spearman.append(spearman_score)
    
    for param in model.parameters():
      param.requires_grad=True
    model.train()
    model_name="double-bert-based-case"
    # print(f"{model}_f{fold}_best")
    # print("=======================================")
    if best_spearman <= spearman_score:
      #torch.save(model.state_dict(), f"{model_name}_f{fold}_best")
      torch.save(model.state_dict(), f"./double/Bce-NoOptbinning2/{model_name}_f{fold}_best")
      best_spearman = spearman_score
      #print(valid_y3[:,:N_BERT_LABEL])
      each_speaman_df = calc_each_spearman(valid_y3[:,:N_BERT_LABEL], valid_pred)
      display(each_speaman_df)

    print(f"fold-{fold} epoch {epoch}: {spearman_score} / loss avg: {ave_loss}")
    
  best_spearman_lst.append(best_spearman)
  losses_lst.append(losses)
  epoch_spearman_lst.append(epoch_spearman)
  lr_lst_lst.append(lr_lst)
  each_speaman_dfs.append(each_speaman_df)

  torch.cuda.empty_cache()

loading configuration file config.json from cache at C:\Users\Lab000/.cache\huggingface\hub\models--bert-base-cased\snapshots\5532cc56f74641d4bb33641f5c76a55d11f846e0\config.json
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

Configuration saved in ./double/token_model_config/config\config.json
loading weights file pytorch_model.bin from cache at C:\Users\Lab000/.cache\huggingface\hub\

  0%|          | 0/2736 [00:00<?, ?it/s]

tensor([[0.8889, 0.5556, 0.0000, 0.0000, 1.0000, 0.6667, 0.8889, 0.5556, 1.0000,
         0.0000, 0.6667, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.6667, 1.0000,
         0.6667, 0.0000, 0.7778],
        [0.7778, 0.3333, 0.0000, 0.0000, 1.0000, 1.0000, 0.5556, 0.5556, 1.0000,
         0.0000, 0.0000, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.6667, 0.0000,
         1.0000, 0.0000, 1.0000]], device='cuda:0')
tensor([[1.0000, 0.3333, 0.0000, 1.0000, 0.3333, 0.0000, 0.6667, 0.6667, 0.0000,
         0.0000, 0.6667, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 1.0000],
        [0.8889, 0.5556, 0.0000, 0.6667, 0.6667, 0.6667, 0.6667, 0.5556, 0.3333,
         0.0000, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.3333, 0.3333,
         1.0000, 0.0000, 0.8889]], device='cuda:0')
tensor([[0.6667, 0.5000, 0.0000, 0.5000, 0.5000, 0.5000, 0.8333, 0.6667, 0.5000,
         0.0000, 0.5000, 0.5000, 0.0000, 0.0000, 0.5000, 0.0000, 0.0000, 0.0000,
         1.0000, 0

KeyboardInterrupt: 